In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from snorkel import SnorkelSession
session = SnorkelSession()
from snorkel.models import  Document, Sentence

In [3]:
from sqlalchemy import  any_,or_,and_

In [4]:
from snorkel.models import Candidate

In [5]:
for k in [0,1,2,3]:
    print 'split %i:%i/%i candidates'%(k,
                                       session.query(Candidate).filter(and_(Candidate.split==k,Candidate.labels.any())).count(),
                                       session.query(Candidate).filter(Candidate.split==k).count()
    )

split 0:0/15299 candidates
split 1:0/9549 candidates
split 2:0/13394 candidates
split 3:0/76867 candidates


In [7]:
from snorkel.models import Candidate, candidate_subclass
REGULATOR = candidate_subclass('REGULATOR', ['Chemical', 'Gene'])

In [35]:
for k in [0,1,2,3]:
    print 'split %i:%i/%i candidates'%(k,
                                       session.query(REGULATOR).filter(and_(REGULATOR.split==k,REGULATOR.labels.any())).count(),
                                       session.query(REGULATOR).filter(REGULATOR.split==k).count()
    )

split 0:0/15299 candidates
split 1:0/9549 candidates
split 2:0/13394 candidates
split 3:0/76867 candidates


In [11]:
c1 = session.query(Candidate).filter(Candidate.split==1).all()[0]

In [15]:
from snorkel.db_helpers import reload_annotator_labels

In [30]:
from snorkel.models.annotation import GoldLabel, Label, StableLabel

In [31]:
session.query(StableLabel).count()

321543

In [20]:
gl1 = session.query(GoldLabel).all()[1]

In [29]:
gl1.candidate_id

5144

In [32]:
# try to reload

In [33]:
from snorkel.db_helpers import reload_annotator_labels


In [34]:
reload_annotator_labels(session, REGULATOR, 'gold', split=0, debug=True)

AnnotatorLabels created: 0
AnnotatorLabels not matched to candidates (split=0): 314262


In [11]:
# get test cands + labels to plug into lstm

In [12]:
from snorkel.annotations import load_gold_labels
test_cands   = session.query(REGULATOR).filter(REGULATOR.split == 2).all()
test_labels = load_gold_labels(session, annotator_name='gold', split=2)

print('Test set:\t{0} candidates'.format(len(test_cands)))

Test set:	13350 candidates


In [ ]:
####

In [15]:
from snorkel.learning import reRNN

lstm = reRNN(seed=1701, n_threads=None,)

Instructions for updating:
Use the retry module or similar alternatives.


In [22]:
lstm.load(model_name='50epochs_batch32_train_test',save_dir="../baseline/checkpoints/")

INFO:tensorflow:Restoring parameters from ../baseline/checkpoints/50epochs_batch32_train_test/50epochs_batch32_train_test-0
[reRNN] Loaded model <50epochs_batch32_train_test>


In [23]:
tp, fp, tn, fn = lstm.error_analysis(session, test_cands, test_labels)

Scores (Un-adjusted)
Pos. class accuracy: 0.211
Neg. class accuracy: 0.805
Precision            0.121
Recall               0.211
F1                   0.154
----------------------------------------
TP: 317 | FP: 2312 | TN: 9538 | FN: 1183

